In [1]:
import sys, os
import numpy as np
from scipy import stats
from iminuit import Minuit

import analysis
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
c = 299792.458 # km/s

v0 = 220/c # unitless dispersion veloctiy 
vObs = 232/c # unitless boost velocity

In [3]:
ma = 2*np.pi*1e6
freqs = np.linspace(1e6-2, 1e6+5, 141)

template = analysis.getAxionTemplate(ma, freqs, v0, vObs)
data = stats.norm.rvs(loc = template+1, scale = 1, size = (10, len(template))) # needs to be at least (1, N)

#### Do the Analysis

In [4]:
# Format the freqs, data, and templates in convenient units
freq_center = (np.amax(freqs) + np.amin(freqs)) / 2
freq_width = np.amax(freqs) - np.amin(freqs)
freqs_no_dim = 2*(freqs-freq_center) / freq_width

data_rescale = np.mean(data)
data_no_dim = data / data_rescale

template_rescale = np.amax(template)
template_no_dim = template / template_rescale

#### Format the Optimizer Guess and Bounds

In [5]:
amp_guess = 0
amp_upper_bound = 2*(np.amax(data) - np.amin(data))
amp_lower_bound = -2*(np.amax(data) - np.amin(data))

mean_guess = np.mean(data_no_dim, axis = 1)
mean_upper_bounds = 2*mean_guess
mean_lower_bounds = np.zeros_like(mean_guess)

slope_guess = np.zeros_like(mean_guess)
slope_upper_bounds = .25*np.ones_like(slope_guess)
slope_lower_bounds = -.25*np.ones_like(slope_guess)

upper_bounds = np.append(amp_upper_bound, np.append(mean_upper_bounds, slope_upper_bounds))
lower_bounds = np.append(amp_lower_bound, np.append(mean_lower_bounds, slope_lower_bounds))
bounds = np.vstack((lower_bounds, upper_bounds)).T
guess = np.append(0, np.append(mean_guess, slope_guess))

#### Run the Optimization

In [6]:
loss = lambda x: analysis.NegLL(x, freqs_no_dim, data_no_dim, template_no_dim)
grad = lambda x: analysis.NegLL_Jac(x, freqs_no_dim, data_no_dim, template_no_dim)

m = Minuit(loss, guess, grad = grad)
m.errordef = 1
m.limits = bounds

# Initially fix all parameters
for i in range(len(guess)):
    m.fixed['x' + str(i)] = True
    
# Fit each of the PSDs under their independent null hypotheses   
for i in range(data_no_dim.shape[0]):
    m.fixed['x' + str(i + 1)] = False
    m.fixed['x' + str(i + 1 + data_no_dim.shape[0])] = False
    
    m.migrad()
    
    m.fixed['x' + str(i + 1)] = True
    m.fixed['x' + str(i + 1 + data_no_dim.shape[0])] = True

Null_TS = m.fval

# Now release all parameters and fit the signal model hypothesis
for i in range(len(guess)):
    m.fixed['x' + str(i)] = False
m.migrad()

Signal_TS = m.fval

Discovery_TS = Null_TS - Signal_TS
Best_Fit = m.values['x0'] / template_rescale * data_rescale
Error = m.errors['x0'] / template_rescale * data_rescale

print(Discovery_TS, Best_Fit, Error)

10.923624983905029 0.6310756556530353 0.19065541951679682
